# Parallel Tempering using pure Python (Multiprocessing)

Parallel tempering is a technique for improving the efficiency of MCMC, especially in the case of multi-modal target distributions. The core idea is to build a sequence of distributions starting with an easy-to-sample reference (e.g. the prior) and ending with the difficult-to-sample target (e.g. the posterior). Given the target $\pi$ and the reference $\pi_0$, this sequence is constructed using a "temperature" parameter $0 < \beta < 1$:
$$\pi_\beta = \pi_0^{1-\beta} \cdot \pi^\beta$$
Here, for $\beta = 0$ (also referred to as "cold" chain) the distribution is equal to the target, for $\beta = 1$ (also referred to as "hot" chain) it is equal to the reference. By running multiple chains with an ascending sequence of temperatures $\beta_0 = 0, \beta_1, \dots, \beta_n = 1$ and allowing theses chains to pass states based on Metropolis-Hastings, the simpler properties of hotter distributions improve the exploration of the state space and can result in faster convergence of the cold chain to the target distribution.

*hopsy* implements parallel tempering. This notebook illustrates it by sampling a mixture of Gaussians that has two distinct modes. Depending on the starting point, vanilla MCMC approaches have trouble capturing the multi-modality. This is because once the chain has found a mode, Metropolis-Hastings proposal are very unlikely to propose high-density points in the other mode. With parallel tempering, the hotter chains are less effected by this and can better sample broadly. By passing these states on to the colder chains, other modes can be explored.

In [1]:
import numpy as np
import hopsy

import matplotlib.pyplot as plt

The sampling code with all imports must be defined within a file or function like here. The code is later called by an MPI process, enabling communication of the tempered chains in *HOPS*. Notice, that for the definition of the Markov chain, a synchronized `RandomNumberGenerator` (must have the same seed for each process!) and an exchange probability must be given.

Important: Parallel tempering works best, if each chain runs on its own core. Therefore, it is not advised to use parallel tempering on machines with low core counts.

In [2]:
class GaussianMixture:
    def __init__(self, mu1, mu2):
        epsilon = 0.05
        cov = epsilon * np.eye(2, 2)
        self.model1 = hopsy.Gaussian(mean=mu1, covariance=cov)
        self.model2 = hopsy.Gaussian(mean=mu2, covariance=cov)

    def log_density(self, x):
        return np.log(
            np.exp(-self.model1.compute_negative_log_likelihood(x))
            + np.exp(-self.model2.compute_negative_log_likelihood(x))
        )


A = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
b = np.array([1, 1, 1, 1])

model = GaussianMixture(np.ones(2).reshape(2, 1), -np.ones(2).reshape(2, 1))
problem = hopsy.Problem(A, b, model)

syncRng = hopsy.RandomNumberGenerator(seed=4321)

mc = hopsy.MarkovChain(
    proposal=hopsy.GaussianCoordinateHitAndRunProposal,
    problem=problem,
    parallelTemperingSyncRng=syncRng,
    exchangeAttemptProbability=0.15,
    starting_point=0.9 * np.ones(2),
    parallel_tempering_backend
)
mc.proposal.stepsize = 0.25

rng = hopsy.RandomNumberGenerator(rank + chain_idx + 11)

acc_rate, samples = hopsy.sample(markov_chains=mc, rngs=rng, n_samples=n_samples)
return acc_rate, samples, rank

RuntimeError: MPI not supported on current platform

In [ ]:
def run_tempered_chains(n_samples: int, n_temps: int, chain_idx: int):
    with ipp.Cluster(engines='mpi', n=n_temps) as rc:
        result = view.apply_sync(run_tempered_chain, n_samples, chain_idx)


result = sorted(result, kdef
run_tempered_chain(n_samples: int, chain_idx: int):
import hopsy
import numpy as np
from mpi4py import MPI

class GaussianMixture:
    def __init__(self, mu1, mu2):
        epsilon = 0.05
        cov = epsilon * np.eye(2, 2)
        self.model1 = hopsy.Gaussian(mean=mu1, covariance=cov)
        self.model2 = hopsy.Gaussian(mean=mu2, covariance=cov)

    def compute_negative_log_likelihood(self, x):
        return -np.log(
            np.exp(-self.model1.compute_negative_log_likelihood(x))
            + np.exp(-self.model2.compute_negative_log_likelihood(x))
        )


A = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
b = np.array([1, 1, 1, 1])

model = GaussianMixture(np.ones(2).reshape(2, 1), -np.ones(2).reshape(2, 1))
problem = hopsy.Problem(A, b, model)

replicates = 4
n_chains = 5
n_cores =  20


assert n_chains>1
temperature_ladder = [1. - index/(n_chains-1) for i in range(n_chains)]
starting_point = .9 * np.ones(2)

mcs = []
rngs = []

for replicate_id in range(replicates):
    # this loops connects chains via shared memory
    syncRng = hopsy.RandomNumberGenerator(replicate_id + 42)
    shared_memory_names = hopsy.create_shared_memory(starting_point.shape)
    mcs += [
        hopsy.MarkovChain(
            proposal=hopsy.GaussianCoordinateHitAndRunProposal,
            problem=problem,
            parallel_tempering_sync_rng=syncRng,
            exchange_attempt_problem=0.15,
            starting_point=starting_pint,
            parallel_tempering_shared_memory=shared_memory_names,
            parallel_tempering_coldness=c,
        )
        for c in temperature_ladder
    ]
    rngs += [
        hopsy.RandomNumberGenerator(42+i,replicate_id)
        for i in range(len(temperature_ladder))
    ]

for mc in mcs:
    mc.proposal.stepsize = 0.25

acc_rate, samples = hopsy.sample(markov_chains=mcs, rngs=rngs, n_samples=n_samples)

In [ ]:
print(hopsy.rhat(samples), hopsy.ess(samples))

The cold chains seem to have converged. The following plot illustrates posteriors of all tempered distributions. The multi-modality is very well captured by all replicas.

In [ ]:
fig, axs = plt.subplots(1, n_replicas, sharey=True, figsize=(20, 5))
for i in range(n_replicas):
    ax = axs[i]
ax.set_title(f"replica {i + 1}")
for j in range(n_temps):
    ax.hist(result[i][j][:, :, 0].flatten(), density=True, alpha=0.245, label="$\\beta_" + str(j) + "$", bins=100)
plt.tight_layout()
plt.legend()
plt.show()